In [42]:
using PyCall
using Statistics
using LinearAlgebra
using Random
using Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
using JLD2
using Tensor_FixedSeedMC
import Tensor_FixedSeedMC as TCIMC

np = pyimport("numpy")
@pyimport tf_quant_finance as tff
@pyimport tensorflow as tf


In [50]:
# 満期・金利・行使価格など
T = 1.0               # 満期
r = 0.01              # 無リスク金利
K = 100.0             # 行使価格
d = 5                 # 原資産数
nPath = 10           # モンテカルロパス数
S0 = fill(100, d)     # 原資産の初期値ベクトル
seed = 1234

function chebyshev_lobatto_nodes(n::Int, a::Real, b::Real)
    # [-1,1] のチェビシェフ-ロバット節点 → [a,b] へ写像
    x_standard = [cos(π * (n - 1 - i) / (n - 1)) for i in 0:n-1]
    x_scaled = 0.5 * (b - a) .* (x_standard .+ 1) .+ a
    return x_scaled, x_standard
end

num_nodes = 100
σs, _ = chebyshev_lobatto_nodes(num_nodes, 0.15, 0.25)    # ボラティリティグリッド
stock0, _ = chebyshev_lobatto_nodes(num_nodes, 90, 120)   # 初期株価グリッド

corrMat = [
    1.0      0.303659  0.505532  0.719655  0.728832;
    0.303659 1.0      0.401077  0.515272  0.178203;
    0.505532 0.401077 1.0      0.132542  0.722767;
    0.719655 0.515272 0.132542 1.0      0.394723;
    0.728832 0.178203 0.722767 0.394723  1.0
]

function calculate_option_price(
    T::Float64, r::Float64, K::Float64, d::Int, S0::Vector{Int}, stock0::Vector{Float64},
    localindex_v::Vector{Int}, nPath::Int, σs::Vector{Float64}, corrMat
)::Tuple{Float64, Float64}
    # ボラリティ、初期株価の組み合わせを抽出
    vols = [σs[i] for i in localindex_v[1:2:2*d]]
    S0s = [stock0[i] for i in localindex_v[2:2:2*d]]

    np.random.seed(seed)  # numpyのシードを固定
    tf.random.set_seed(seed)  # tensorflowのシードを固定
    
    process = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        volatilities=vols,
        corr_matrix=corrMat)
    
    paths = process.sample_paths(
        times=[T],
        initial_state=S0s,
        random_type=tff.math.random.RandomType.PSEUDO,
        seed = seed,
        num_samples=nPath
    ).numpy()
    
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    mean_pv = exp(-r * T) * mean(payoffs)
    std_pv = exp(-r * T) * std(payoffs)
    return (mean_pv, std_pv)
end


calculate_option_price (generic function with 1 method)

In [51]:
# 各資産ともグリッドの最大値を使う場合
localindex_v = [100 for _ in 1:2d]

mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
println("Present Value: ", mean_pv)
println("Standard Deviation: ", std_pv)
println("Abs.Err.: ", std_pv / sqrt(nPath))


Present Value: 6.16224477441282
Standard Deviation: 9.312806162122191
Abs.Err.: 2.944967887995743


In [ ]:
using Random

d = 5
nPath = 100
num_grid = 100

# 誤差を記録する配列
abs_errors = Float64[]

Random.seed!(seed)
for _ in 1:100
    # [σ1, S0_1, σ2, S0_2, ..., σd, S0_d] をランダムに生成
    localindex_v = [rand(1:num_grid) for _ in 1:2d]
    mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
    abs_err = std_pv / sqrt(nPath)
    push!(abs_errors, abs_err)
end

println("Mean Abs.Err.: ", mean(abs_errors))
println("Max  Abs.Err.: ", maximum(abs_errors))


In [53]:
abo(localindex_v) = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)[1]
localdims_v = fill(num_nodes, 2d)
firstpivot_v = rand(1:num_nodes, 2d)
phi = TCI.CachedFunction{Float64}(abo, localdims_v)


# TCIワンショット補間関数
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)
    for _ in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot)
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end
    qtt, ranks, errors = TCI.crossinterpolate2(Float64, func, localdims, [firstpivot],
                                                  tolerance=tol, maxiter=6, verbosity=1,
                                                  loginterval=1, pivotsearch=:rook, maxbonddim=30)
    return qtt, errors
end

tci_oneshot (generic function with 1 method)

In [ ]:
tol_mc = 1e-3
tol = tol_mc

tci_time = @elapsed begin
    tt_tci, errors_asianbarrier = tci_oneshot(phi, d, localdims_v, firstpivot_v, tol)
end

In [36]:
function generate_random_combinations(len, d, num_samples)
    Random.seed!(1234)

    possible_values = collect(1:len)
    random_combinations = Vector{Int}[]
    
    for _ in 1:num_samples
        combination = rand(possible_values, d)
        push!(random_combinations, combination)
    end
    
    return random_combinations
end


# Option pricing taking random combinations of parameters
function rand_inner(T, r, K, d, S0, stock0, nPath, σs, corrMat, len, num_samples, random_combinations)
    results = Vector{ComplexF64}()
    for i in 1:num_samples
        ind = random_combinations[i]
        localindex_v = ttog.indices_1d_to_2d(ind, 100)
        result, err = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
        push!(results, result)
    end
    return results, random_combinations
end

rand_inner (generic function with 1 method)

In [37]:
num_samples = 100  # Number of parameter combinations
len

UndefVarError: UndefVarError: `len` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [38]:
random_combinations = generate_random_combinations(len, d, num_samples);

UndefVarError: UndefVarError: `len` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [39]:
results, combinations = rand_inner(T, r, K, d, S0, stock0, nPath, σs, corrMat, len, num_samples, random_combinations)
#results, combinations = rand_inner(T, r, K, d, S0, stock0, localindex_v, nPath, σs, reordered_corrMat, len, num_samples, random_combinations)

mc = real(results)
#JLD2.@save "vol&stock0_mc_d5_50000000_const_extended.jld2" mc
mc

UndefVarError: UndefVarError: `len` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [40]:
mc

UndefVarError: UndefVarError: `mc` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [41]:
#JLD2.@save "vol&stock0_mc_d5_50000000_rand_reordered.jld2" mc
#JLD2.@save "vol&stock0_mc_d5_50000000_rand_cheby.jld2" mc
JLD2.@save "vol&stock0_mc_d5_100000000_const_noise_cheby_gr100.jld2" mc

UndefVarError: UndefVarError: `mc` not defined in `Main`
Suggestion: check for spelling errors or missing imports.